In [1]:
from amplify import gen_symbols, BinaryPoly, IsingPoly
from amplify import sum_poly
from amplify.constraint import equal_to
import matplotlib.pyplot as plt
from amplify import Solver
from amplify.client import FixstarsClient
import matplotlib.ticker as ticker   
import os

In [2]:
def q_annealing(n, c):
    s = gen_symbols(IsingPoly, n+1, n+1)

    # 確定しているもののスピンを確定------------------------------------------
    for i in range(n):
        for j in range(n):
            if c[i][j] == "B":
                s[i][j] = 1
            if c[i][j] == "W":
                s[i][j] = -1

    # ハミルトニアン1 コスト項　反強磁性イジングモデル-----------------------------------
    f = 0
    for i in range(n):
        for j in range(n):
            if i != n-1:
                f += s[i][j] * s[i+1][j]
            if j != n-1:
                f += s[i][j] * s[i][j+1]

    # n=1のときや？が存在しないケースに対応するためにまったく関係ないスピンを追加
    # スピンは必ず-1になるので+1を足して打ち消し
    f += s[n][n] + 1 
                
    model = f  
    
    # 計算実行------------------------------------------------------------------------------------------
    client = FixstarsClient()
    with open("tokenfile.txt", "r") as f:
        token = f.readline()
    client.token = token
    client.parameters.timeout = 9000  # タイムアウト9秒
    solver = Solver(client)

    result = solver.solve(model)
    if len(result) == 0:
        raise RuntimeError("Any one of constraints is not satisfied.")

    energy, values = result[0].energy, result[0].values
    
    return int((2*n*(n-1) -energy)/2)

In [ ]:

# テストケース読み込み
# https://www.dropbox.com/sh/nx3tnilzqz7df8a/AACpUzqOotp665-GQMdAbHJua/ABC193/F?dl=0&subfolder_nav_tracking=1

files = os.listdir("ABC193F/in")
for file in files:
    with open("ABC193F/in/" + file, "r") as f:
        n = int(f.readline())
        c = [f.readline() for i in range(n)] 
        
    with open("ABC193F/out/" + file, "r") as f:
        ans = int(f.readline())

    try:
        result = q_annealing(n, c)
        print(file, "Amplify結果 =", result, "正解 =", ans)
    except:
        pass
